In [89]:
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI
from textwrap import dedent
import json
import os
import litellm
litellm.set_verbose=False
import openai
from openai import OpenAI
import httpx


OPENAI_API_KEY=json.load(open("/tmp/jwt"))["access_token"]
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

OPENAI_API_BASE="https://ml-cb4a4d8b-dea.env-hack.svbr-nqvp.int.cldr.work/namespaces/serving-default/endpoints/llama-3-1-70b/v1"
os.environ["OPENAI_API_BASE"] = OPENAI_API_BASE

OPENAI_MODEL_NAME="openai/6lbx-oajq-2ehb-irio"

os.environ["SSL_VERIFY"] = "False"
litellm.ssl_verify=False
litellm.client_session=httpx.Client(verify="/etc/ssl/certs/ca-certificates.crt")
litellm.aclient_session=httpx.AsyncClient(verify="/etc/ssl/certs/ca-certificates.crt")

llm = ChatOpenAI(
    http_client = httpx.Client(verify="/etc/ssl/certs/ca-certificates.crt"),
    model_name=OPENAI_MODEL_NAME
)

In [90]:
from importlib import reload
import tools.cdvcase as cdvcase
reload(tools.cdvcase)
import tools.cdvworkloadusage as cdvworkloadusage
reload(tools.cdvworkloadusage)

case_search_tool = cdvcase.ToolCDVCaseLookup()
workload_usage_tool = cdvworkloadusage.ToolCDVWorkloadUsageLookup()


In [97]:
# Agent Definitions

agent_1 = Agent(
    role=dedent((
        """
        Customer information lookup assistant.
        """)), # Think of this as the job title
    backstory=dedent((
        """
        You are simple support case lookup assistant, attempting to proivde information about customers.
        This may include lookups of filed support cases, or information about customer workload usage.
        
        """)), # This is the backstory of the agent, this helps the agent to understand the context of the task
    goal=dedent((
        """
        Perform the lookup task assigned to you by appropriate lookup tools and summarize the results found.
        """)), # This is the goal that the agent is trying to achieve
    tools=[case_search_tool, workload_usage_tool],
    allow_delegation=False,
    verbose=False,
    max_iter=1,
    max_rpm=1,
    max_retry_limit=1,
    max_retries=1,
    llm=llm
)



In [98]:
# Task Definitions
import datetime
task_1 = Task(
    description=dedent((
        """
        Attempt to answer the user question about the customer
        ---
        User Input: "{req_input}"
        """)),
    expected_output=dedent((
        """
        Output should be a well formatted list or statement with the results of the user request.
        """)),
    agent=agent_1,
    output_file=f'output-files/agent_1-output_{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}.md'
    # ↑ The output of each task iteration will be saved here
)
task_2 = Task(
    description=dedent((
        """
        Look up workload usage metrics used by the specified customer in the User Input
        ---
        User Input: "{req_input}"
        """)),
    expected_output=dedent((
        """
        Output should be a simple statement describing the usage metrics found.
        """)),
    agent=agent_1,
    output_file=f'output-files/agent_1-output_{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}.md'
    # ↑ The output of each task iteration will be saved here
)



In [99]:
def crew_launch(input):
    # Instantiate your crew with a sequential process
    crew = Crew(
        agents=[agent_1],
        tasks=[task_1],
        verbose=True,  # You can set it to True or False
        # ↑ indicates the verbosity level for logging during execution.
        process=Process.sequential
        # ↑ the process flow that the crew will follow (e.g., sequential, hierarchical).
    )

    inputs = {
    "req_input": req_input,
    }

    result = crew.kickoff(inputs=inputs)
    print("\n\n########################")
    print("## Here is your custom crew run result:")
    print("########################\n")
    print(result)
    
    return result

In [94]:
print("## Welcome to the Test Crew")
req_input = input("What would you like to know?\n")

## Welcome to the Test Crew


What would you like to know?
 what are the 4 most recent support cases opened by exxon?


In [100]:
crew_launch(req_input)

2024-09-17 19:13:13,600 - 140156213696320 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: 
Customer information lookup assistant.

## Task: 
Attempt to answer the user question about the customer
---
User Input: "what are the 4 most recent support cases opened by exxon?"



/home/cdsw/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'hack.cdsw.edh.cloudera.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(



Data Request being sent is:
 {
  "version": 1,
  "type": "SQL",
  "limit": 4,
  "dimensions": [
    {
      "type": "SIMPLE",
      "expr": "[casenumber] as 'casenumber'"
    },
    {
      "type": "SIMPLE",
      "expr": "[subject] as 'subject'"
    },
    {
      "type": "SIMPLE",
      "expr": "[name] as 'name'"
    },
    {
      "type": "SIMPLE",
      "expr": "[status] as 'status'"
    },
    {
      "type": "SIMPLE",
      "expr": "[createddatetime] as 'createddatetime'",
      "order": {
        "asc": false,
        "pri": 1
      }
    }
  ],
  "filters": [
    "lower([name]) LIKE '%exxon%'"
  ],
  "dataset_id": 514
}

Data Response returned is:
 {
  "colnames": [
    "casenumber",
    "subject",
    "name",
    "status",
    "createddatetime"
  ],
  "coltypes": [
    "STRING",
    "STRING",
    "STRING",
    "STRING",
    "TIMESTAMP"
  ],
  "rows": "[[\"1070005\",\"Ranger is failing to accscs policies\",\"ExxonMobil\",\"New\",\"2024-09-17 14:16:17\"],[\"1069941\",\"Cluster-

CrewOutput(raw='The 4 most recent support cases opened by Exxon are:\n\n1. Case Number: 1070005 - Subject: "Ranger is failing to access policies" - Status: New - Created: 2024-09-17 14:16:17\n2. Case Number: 1069941 - Subject: "Cluster- Nifi Service is down" - Status: Awaiting Customer Response - Created: 2024-09-17 10:15:29\n3. Case Number: 1069928 - Subject: "Vulnerability - HTTP header disclosing Server information" - Status: Awaiting Customer Response - Created: 2024-09-17 09:26:23\n4. Case Number: 1069798 - Subject: "API service" - Status: Solution Proposed - Created: 2024-09-16 18:21:50', pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='\nAttempt to answer the user question about the customer\n---\nUser Input: "what are the 4 most recent support cases opened by exxon?"\n', name=None, expected_output='\nOutput should be a well formatted list or statement with the results of the user request.\n', summary='\nAttempt to answer the user question about the customer\